<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

In [11]:
train_url = 'https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/WikiLarge_Train.csv'

In [48]:
train_df = pd.read_csv(train_url)

In [33]:
train_df.iloc[4].values

array(['Geneva -LRB- , ; , ; , ; ; -RRB- is the second-most-populous city in Switzerland -LRB- after Zürich -RRB- and is the most populous city of Romandie -LRB- the French-speaking part of Switzerland -RRB- .',
       1,
       'Geneva  , ; , ; , ; ;  is the second-most-populous city in Switzerland  after Zürich  and is the most populous city of Romandie  the French-speaking part of Switzerland  .'],
      dtype=object)

In [17]:
train_df.sample(20)[['original_text', 
                     'label']].style.set_properties(subset=['original_text'], 
                                                    **{'width': '500px'})

,original_text,label
275869,"In 1991 , they started in Boston , Massachusetts .",0
238423,"Korkeasaari is an island in Helsinki , Finland .",0
279771,The Republic of Namibia is a country in southern Africa on the Atlantic coast .,0
360192,"Bryan Adams -LRB- November 9 , 1959 -RRB- is a Canadian rock singer .",0
159516,They soon split into cat-like and dog-like forms -LRB- Feliformia and Caniformia -RRB- .,1
227805,Other websites,0
278723,"So , Hyderabad came into the Telugu speaking community and thus became the capital of Andhra Pradesh .",0
205468,"The club 's home is the 22,250 capacity Headingley Stadium , which is in the suburb of Headingley , northwest Leeds , where they have played since 1890 .",1
130239,"Gushank , p. 251 ; Varley , Paul .",1
77262,"England , Northern Ireland , Scotland and Wales have discrete systems of education .",1


In [ ]:
# Observations of things that need cleaning:
# join " 's " with their associated words; same with contractions  - done
# Some sentences are partials of other sentences within the corpus
# Need to address accents - not sure if I do
# Need to address punctuation (vectorizers) - done through regex
# Address weird quotations - (example at index 11005) '' yogurt ' ''
# Need to address -LRB and -RRB which are lemma references to left 
# and right parentheses - done
# remove 'â' (misformatting) - this is incorrect encoding. use ftfy package
# 


# Observations
  # difficulty can be a combination of hard words, hard to pronounce or unfamiliar
  # names, long-run on sentences, harder topics (eg Linux kernel or
  # referring to dog breeds as an example) or just non-sensical sentences without
  # context




In [2]:
# Found a solution to help with contractions
!pip install contractions
!pip install gensim
!pip install ftfy
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 6.9 MB/s 
     |████████████████████████████████| 106 kB 56.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 236 kB 6.8 MB/s 


In [100]:
import re
import contractions
from gensim.utils import simple_preprocess
import ftfy
from unidecode import unidecode

def text_processing(s):
  contract_lst = {'\'ve': '\'ve',
                ' \'ll': '\'ll',
                ' \'d': '\'d',
                ' n\'t': 'n\'t'}
  for key,value in contract_lst.items():
    # remove spaces between contractions
    s = re.sub(key, value,s)
  
  # replace contractions
  s = contractions.fix(s)

  # remove empty quotes
  s = re.sub('\'\'','',s)

  # remove possessive "s"
  s = re.sub(' \'s','',s)


  # remove lrb and rrb references
  pattern = r'(-LRB-|-RRB-)+'
  s = re.sub(pattern,'',s)


  # remove ndash
  s = re.sub('\sndash\s','',s)

  # fix issues with incorrect encoding
  s = ftfy.fix_text(s)
  
  # remove punctuation and symbols
  s = re.sub('[$,.!?;:%@&\/\\\]*','',s)

  # remove numbers 
  s = re.sub('[0-9]*','',s)


  # positive lookbehind for cases where dashes proceeded by spaces or
  # other dashes (not suggesting a hyphenated word or name)
  s = re.sub('(?<=[ -])-','',s)


  return s
  # return simple_preprocess(s)



In [97]:
text = 'SOS -LRB- Â Â Â â '' â '' â '' Â Â Â -RRB- is a Morse code . It is used as distress code , to signal danger .'
fixed = text_processing(text)
fixed

'SO IS  Â Â Â â  â  â  Â Â Â  is a Morse code  It is used as distress code  to signal danger '

In [101]:
# %%timeit
train_df['processed_text'] = train_df['original_text'].apply(text_processing)

In [106]:
train_df[['original_text','processed_text']].sample(20).style.set_properties(subset=['original_text'], 
                                                    **{'width': '500px'})

,original_text,processed_text
190982,Salts that hydrolyze to produce hydroxide ions when dissolved in water are basic salts and salts that hydrolyze to produce hydronium ions in water are acid salts .,Salts that hydrolyze to produce hydroxide ions when dissolved in water are basic salts and salts that hydrolyze to produce hydronium ions in water are acid salts
318305,"White dwarfs are not very bright because they are smaller than many brighter stars - not because they are cold . Some white dwarfs are blue , instead of white .",White dwarfs are not very bright because they are smaller than many brighter stars not because they are cold Some white dwarfs are blue instead of white
243421,The song of the black-capped chickadee is a clear whistle .,The song of the black-capped chickadee is a clear whistle
320045,The length of time it took to record the song created tension between the Beatles .,The length of time it took to record the song created tension between the Beatles
24647,"In Duck Hunt , players utilize the Nintendo Zapper Light Gun that must be plugged into their NES consoles , and attempt to shoot down either ducks or clay pigeons in mid-flight .",In Duck Hunt players utilize the Nintendo Zapper Light Gun that must be plugged into their NES consoles and attempt to shoot down either ducks or clay pigeons in mid-flight
221455,in computer games it is called a combo .,in computer games it is called a combo
387985,"Also during 1992 , shortly after releasing this , Rancid were signed to Bad Religion guitarist Brett Gurewitz 's label , Epitaph Records , and finally released their first album in 1993 , which is also self-titled .",Also during shortly after releasing this Rancid were signed to Bad Religion guitarist Brett Gurewitz label Epitaph Records and finally released their first album in which is also self-titled
105084,"Rage Against the Machine is noted for its innovative blend of alternative rock , punk rock , rap , heavy metal and funk as well as its revolutionary politics and lyrics .",Rage Against the Machine is noted for its innovative blend of alternative rock punk rock rap heavy metal and funk as well as its revolutionary politics and lyrics
377155,Today the flowers are national symbols of Korea .,Today the flowers are national symbols of Korea
15128,"Currently , EA 's most successful products are sports games published under its EA Sports label , games based on popular movie licenses such as Harry Potter and games from long-running franchises like Need for Speed , Medal of Honor , The Sims , Battlefield and the later games in the Burnout and Command & Conquer series .",Currently EA most successful products are sports games published under its EA Sports label games based on popular movie licenses such as Harry Potter and games from long-running franchises like Need for Speed Medal of Honor The Sims Battlefield and the later games in the Burnout and Command Conquer series


In [47]:
train_df[train_df['processed_text'].str.contains('The ball is about 3\xa01/4 inches  83 centimetres  in diameter and about four ounces  1134 grams')]

,original_text,label,formatted_text,processed_text
304503,The ball is about 3 1/4 inches -LRB- 8.3 centi...,0,The ball is about 3 1/4 inches 8.3 centimetre...,The ball is about 3 1/4 inches 83 centimetres...


In [104]:
train_df.iloc[110574][['original_text','processed_text']].values

array(["The appellation Driekoningen -LRB- the Three Kings -RRB- is also often found in 17th - and 18th-century Dutch star charts and seaman 's guides .",
       'The appellation Driekoningen  the Three Kings  is also often found in th  and th-century Dutch star charts and seaman guides '],
      dtype=object)